In [555]:
import seaborn as sns
import pandas as pd
import numpy as np
sns.set()

df = pd.read_csv('../outputs/db.csv', index_col=0)

df_2017 = df.loc[((df['Year'] == 2016) & (df['Subject'] == 'All')), ['Country','GDP per capita (World Bank)', 'GDP per capita (PWT)', 'Documents', 'H index', 'Citations', 'Self-citations', 'Population', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_2017_economic = df.loc[((df['Year'] == 2017) & (df['Subject'] == 'All')), ['Country','GDP per capita (World Bank)', 'GDP per capita (PWT)']]

df_2007_science = df.loc[((df['Year'] == 2007) & (df['Subject'] == 'All')), ['Country','Documents', 'H index', 'Citations', 'Self-citations', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_2002_science = df.loc[((df['Year'] == 2002) & (df['Subject'] == 'All')), ['Country','Documents', 'H index', 'Citations', 'Self-citations', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_1996_science = df.loc[((df['Year'] == 1996) & (df['Subject'] == 'All')), ['Country','Documents', 'H index', 'Citations', 'Self-citations', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_2007_2017 = pd.merge(df_2017_economic, df_2007_science, on="Country")

df_2002_2017 = pd.merge(df_2017_economic, df_2002_science, on="Country")

df_1996_2017 = pd.merge(df_2017_economic, df_1996_science, on="Country")


for i, column in enumerate(df.columns.values):
    print(f"{i} - {column}")

0 - Independent
1 - Sink OFC
2 - Conduit OFC
3 - Code
4 - Country
5 - Year
6 - Population
7 - Population 25 Years Before
8 - Population 25+
9 - Documents
10 - Citable documents
11 - Citations
12 - Self-citations
13 - Citations per Document
14 - H index
15 - Subject
16 - Avg. Adult Schooling
17 - GDP per capita (World Bank)
18 - GDP per capita (PWT)
19 - MAX TAMS
20 - REAL TAMS
21 - Documents / M-TAMS
22 - Documents / R-TAMS
23 - Citations / M-TAMS
24 - Citations / R-TAMS
25 - Gov. Effectiveness
26 - Charges for the use of intellectual property, payments (BoP, current US$)
27 - Charges for the use of intellectual property, receipts (BoP, current US$)
28 - Firms that spend on R&D (% of firms)
29 - High-technology exports (current US$)
30 - Industrial design applications, nonresident, by count
31 - Industrial design applications, resident, by count
32 - Labor force with advanced education (% of total working-age population with advanced education)
33 - Patent applications, nonresidents
34

In [556]:
df_2017

t = df_2017.groupby(by=[df_2017.index, 'Country']).max()
t.sort_values(by="Documents / R-TAMS", ascending=False, inplace=True)


t.head(50)
df_2017.corr()

,GDP per capita (World Bank),GDP per capita (PWT),Documents,H index,Citations,Self-citations,Population,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
GDP per capita (World Bank),1.000000,0.974968,0.246943,0.519242,0.282154,0.171051,-0.041301,-0.057769,0.604025,0.032778,0.039707,0.855688,0.734006,0.749989,0.556995,0.749609
GDP per capita (PWT),0.974968,1.000000,0.266308,0.542492,0.303658,0.183350,-0.053902,-0.004000,0.667455,0.037197,0.045438,0.861729,0.768296,0.788512,0.695071,0.783857
Documents,0.246943,0.266308,1.000000,0.771083,0.989966,0.968331,0.612183,-0.048880,0.235921,0.762017,0.772431,0.189822,0.237619,0.108157,0.118806,0.284957
H index,0.519242,0.542492,0.771083,1.000000,0.800133,0.627904,0.311711,-0.084291,0.499358,0.430280,0.435179,0.530011,0.637111,0.384004,0.400226,0.581937
Citations,0.282154,0.303658,0.989966,0.800133,1.000000,0.956914,0.518694,-0.042330,0.259129,0.675699,0.688003,0.235805,0.288200,0.152177,0.162871,0.312237
Self-citations,0.171051,0.183350,0.968331,0.627904,0.956914,1.000000,0.610672,-0.032582,0.156448,0.767269,0.780134,0.112274,0.143259,0.058828,0.066686,0.192276
Population,-0.041301,-0.053902,0.612183,0.311711,0.518694,0.610672,1.000000,-0.050476,-0.047270,0.958140,0.955093,-0.056267,-0.050224,-0.077013,-0.074254,0.025219
Citations per Document,-0.057769,-0.004000,-0.048880,-0.084291,-0.042330,-0.032582,-0.050476,1.000000,-0.009660,-0.056099,-0.056627,-0.023533,-0.020767,0.327337,0.543381,0.044362
Avg. Adult Schooling,0.604025,0.667455,0.235921,0.499358,0.259129,0.156448,-0.047270,-0.009660,1.000000,0.057887,0.064717,0.522175,0.585272,0.438139,0.437212,0.739316
MAX TAMS,0.032778,0.037197,0.762017,0.430280,0.675699,0.767269,0.958140,-0.056099,0.057887,1.000000,0.999422,-0.013627,0.003773,-0.048989,-0.042667,0.113732


In [557]:
df_2007_2017.corr()

,GDP per capita (World Bank),GDP per capita (PWT),Documents,H index,Citations,Self-citations,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
GDP per capita (World Bank),1.000000,0.974486,0.265263,0.524608,0.273849,0.195666,0.091981,0.617670,0.056365,0.061073,0.739817,0.677356,0.636161,0.595721,0.739662
GDP per capita (PWT),0.974486,1.000000,0.289832,0.548104,0.299610,0.213938,0.109333,0.679893,0.067032,0.071347,0.781479,0.732480,0.685971,0.653958,0.761461
Documents,0.265263,0.289832,1.000000,0.789069,0.972355,0.947703,0.078321,0.262900,0.659538,0.666483,0.281380,0.308969,0.263364,0.278507,0.294971
H index,0.524608,0.548104,0.789069,1.000000,0.770414,0.642674,0.183388,0.514897,0.472439,0.472442,0.655736,0.705386,0.625389,0.651002,0.588355
Citations,0.273849,0.299610,0.972355,0.770414,1.000000,0.973529,0.107286,0.265204,0.490130,0.496807,0.301100,0.329035,0.293056,0.309098,0.295397
Self-citations,0.195666,0.213938,0.947703,0.642674,0.973529,1.000000,0.070927,0.185187,0.511842,0.520611,0.185144,0.203241,0.180896,0.191323,0.199738
Citations per Document,0.091981,0.109333,0.078321,0.183388,0.107286,0.070927,1.000000,-0.042675,-0.050094,-0.046701,0.306371,0.325471,0.460115,0.480075,0.202250
Avg. Adult Schooling,0.617670,0.679893,0.262900,0.514897,0.265204,0.185187,-0.042675,1.000000,0.086101,0.089210,0.558549,0.577888,0.482201,0.487549,0.699226
MAX TAMS,0.056365,0.067032,0.659538,0.472439,0.490130,0.511842,-0.050094,0.086101,1.000000,0.999245,0.031435,0.044107,0.018187,0.024844,0.110301
REAL TAMS,0.061073,0.071347,0.666483,0.472442,0.496807,0.520611,-0.046701,0.089210,0.999245,1.000000,0.040880,0.048820,0.025140,0.029427,0.121574


In [558]:
df_2002_2017.corr()

,GDP per capita (World Bank),GDP per capita (PWT),Documents,H index,Citations,Self-citations,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
GDP per capita (World Bank),1.000000,0.974486,0.268960,0.521514,0.252619,0.187736,0.195284,0.587510,0.058256,0.060141,0.700489,0.650888,0.609417,0.581515,0.730149
GDP per capita (PWT),0.974486,1.000000,0.296096,0.548104,0.277931,0.206637,0.193945,0.652228,0.074085,0.075533,0.739893,0.706634,0.660170,0.642255,0.752140
Documents,0.268960,0.296096,1.000000,0.778092,0.983271,0.947797,0.117562,0.281967,0.544995,0.547117,0.305769,0.328215,0.319296,0.329433,0.297231
H index,0.521514,0.548104,0.778092,1.000000,0.722855,0.606991,0.177812,0.507543,0.484298,0.481444,0.684832,0.723050,0.676901,0.692119,0.602785
Citations,0.252619,0.277931,0.983271,0.722855,1.000000,0.980092,0.133911,0.256299,0.434661,0.438571,0.293853,0.314276,0.319039,0.328708,0.275535
Self-citations,0.187736,0.206637,0.947797,0.606991,0.980092,1.000000,0.103007,0.190619,0.429287,0.435511,0.193018,0.205177,0.219752,0.224768,0.189482
Citations per Document,0.195284,0.193945,0.117562,0.177812,0.133911,0.103007,1.000000,0.131734,-0.023343,-0.022013,0.377962,0.402903,0.456027,0.471072,0.338912
Avg. Adult Schooling,0.587510,0.652228,0.281967,0.507543,0.256299,0.190619,0.131734,1.000000,0.103375,0.104238,0.523343,0.553001,0.478252,0.493062,0.661122
MAX TAMS,0.058256,0.074085,0.544995,0.484298,0.434661,0.429287,-0.023343,0.103375,1.000000,0.999218,0.045269,0.054538,0.046415,0.049916,0.090549
REAL TAMS,0.060141,0.075533,0.547117,0.481444,0.438571,0.435511,-0.022013,0.104238,0.999218,1.000000,0.047693,0.056146,0.048323,0.052305,0.097680


In [559]:
df_1996_2017.corr()

,GDP per capita (World Bank),GDP per capita (PWT),Documents,H index,Citations,Self-citations,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
GDP per capita (World Bank),1.000000,0.974486,0.258300,0.522138,0.239465,0.184118,0.161882,0.566240,0.066807,0.067731,0.696273,0.622506,0.568135,0.532795,0.721329
GDP per capita (PWT),0.974486,1.000000,0.285706,0.550561,0.264582,0.203392,0.190591,0.634859,0.086730,0.087309,0.740583,0.682958,0.631734,0.602667,0.732991
Documents,0.258300,0.285706,1.000000,0.751208,0.987571,0.956982,0.124887,0.293937,0.547765,0.543222,0.299829,0.336379,0.324668,0.337944,0.267035
H index,0.522138,0.550561,0.751208,1.000000,0.696231,0.592197,0.178557,0.521217,0.520351,0.513540,0.655404,0.720441,0.663955,0.684717,0.569949
Citations,0.239465,0.264582,0.987571,0.696231,1.000000,0.984215,0.137557,0.268736,0.471632,0.468846,0.286483,0.318983,0.321408,0.333057,0.246740
Self-citations,0.184118,0.203392,0.956982,0.592197,0.984215,1.000000,0.109277,0.212497,0.459463,0.458642,0.197246,0.218835,0.231224,0.238529,0.174588
Citations per Document,0.161882,0.190591,0.124887,0.178557,0.137557,0.109277,1.000000,0.183027,-0.023145,-0.018584,0.431293,0.499441,0.551355,0.582903,0.372733
Avg. Adult Schooling,0.566240,0.634859,0.293937,0.521217,0.268736,0.212497,0.183027,1.000000,0.118640,0.119795,0.506701,0.544441,0.474669,0.486802,0.659727
MAX TAMS,0.066807,0.086730,0.547765,0.520351,0.471632,0.459463,-0.023145,0.118640,1.000000,0.999210,0.048449,0.069167,0.060327,0.068953,0.074819
REAL TAMS,0.067731,0.087309,0.543222,0.513540,0.468846,0.458642,-0.018584,0.119795,0.999210,1.000000,0.051174,0.068432,0.061752,0.068995,0.078918


In [560]:
df_96_17 = df.loc[((df['Year'] <= 2017) & (df['Sink OFC'] == "No") & (df['Subject'] == 'All')), ['Documents', 'H index', 'Citations', 'Self-citations', 'Population', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'GDP per capita (World Bank)', 'GDP per capita (PWT)', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_96_17.corr()

,Documents,H index,Citations,Self-citations,Population,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,GDP per capita (World Bank),GDP per capita (PWT),Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
Documents,1.000000,0.741476,0.870980,0.861171,0.465269,0.020314,0.279007,0.681017,0.688935,0.259950,0.294910,0.263261,0.293465,0.225262,0.235688,0.314715
H index,0.741476,1.000000,0.742689,0.623543,0.305394,0.130091,0.539592,0.451235,0.453049,0.512777,0.545684,0.595470,0.658354,0.604410,0.628825,0.677154
Citations,0.870980,0.742689,1.000000,0.975010,0.282507,0.101289,0.270241,0.476319,0.483131,0.267074,0.297627,0.253624,0.284604,0.291907,0.306916,0.321562
Self-citations,0.861171,0.623543,0.975010,1.000000,0.317620,0.070079,0.193440,0.511103,0.519446,0.190108,0.215374,0.154477,0.174527,0.182596,0.192426,0.217534
Population,0.465269,0.305394,0.282507,0.317620,1.000000,-0.065270,-0.030804,0.930745,0.927817,-0.051937,-0.044083,-0.044552,-0.041795,-0.050838,-0.049616,0.027822
Citations per Document,0.020314,0.130091,0.101289,0.070079,-0.065270,1.000000,-0.033286,-0.062928,-0.060077,0.060122,0.068013,0.138422,0.167179,0.365949,0.410253,0.163291
Avg. Adult Schooling,0.279007,0.539592,0.270241,0.193440,-0.030804,-0.033286,1.000000,0.104724,0.105407,0.548475,0.580337,0.543960,0.586930,0.477613,0.488746,0.684172
MAX TAMS,0.681017,0.451235,0.476319,0.511103,0.930745,-0.062928,0.104724,1.000000,0.999305,0.039422,0.050110,0.022818,0.035673,0.008606,0.014186,0.118829
REAL TAMS,0.688935,0.453049,0.483131,0.519446,0.927817,-0.060077,0.105407,0.999305,1.000000,0.045505,0.056777,0.030574,0.040695,0.014008,0.018317,0.124212
GDP per capita (World Bank),0.259950,0.512777,0.267074,0.190108,-0.051937,0.060122,0.548475,0.039422,0.045505,1.000000,0.968790,0.728324,0.647092,0.624639,0.562309,0.692837


In [561]:
df

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
Country code,,,,,,,,,,,,,,,,,,,,,
8,Yes,No,No,ALB,Albania,1996,3098.699,2202.186,1553.434,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2094.0
12,Yes,No,No,DZA,Algeria,1996,29266.415,14872.253,11543.461,34,...,NaN,150.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,4214.0
16,No,No,No,ASM,American Samoa,1996,54.209,27.982,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Yes,Yes,No,AND,Andorra,1996,64.363,25.571,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2449.0
32,Yes,No,No,ARG,Argentina,1996,35246.376,24259.564,18720.272,798,...,NaN,4012.0,1097.0,0.41749,NaN,NaN,NaN,NaN,NaN,47972.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2019,28515.829,21474.553,16060.699,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,Yes,No,No,VNM,Viet Nam,2019,96462.108,73651.220,60449.404,84,...,87.152702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
887,Yes,No,No,YEM,Yemen,2019,29161.922,14297.617,11664.767,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [562]:
df.describe()

,Year,Population,Population 25 Years Before,Population 25+,Documents,Citable documents,Citations,Self-citations,Citations per Document,H index,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
count,108864.000000,1.088640e+05,1.088640e+05,1.041630e+05,108864.000000,108864.000000,1.088640e+05,1.088640e+05,108864.000000,108864.000000,...,41276.000000,63030.000000,6.077500e+04,51500.000000,40469.000000,82286.000000,31096.000000,38726.000000,3.874800e+04,6.945600e+04
mean,2008.234384,4.027941e+04,2.813672e+04,2.335717e+04,1456.645301,1374.570363,2.838025e+04,8.403505e+03,20.187381,97.051165,...,79.289416,6326.449056,1.283683e+04,0.978749,2055.429538,11500.496374,561.088916,14287.686619,5.048314e+04,2.803261e+04
std,6.882882,1.445740e+05,1.041400e+05,8.627816e+04,12098.637544,11034.953326,2.836377e+05,1.194968e+05,34.383371,143.817760,...,6.469331,25150.193318,7.240889e+04,0.952445,1903.900072,42832.863561,638.398278,21515.787209,2.928430e+05,9.095147e+04
min,1996.000000,7.830000e-01,7.080000e-01,3.305700e+01,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,39.970699,1.000000,1.000000e+00,0.005440,5.911830,0.000000,0.125990,34.000000,1.000000e+00,1.000000e+00
25%,2002.000000,2.600972e+03,1.777932e+03,1.738964e+03,4.000000,4.000000,4.600000e+01,3.000000e+00,5.500000,20.000000,...,75.948097,53.000000,4.400000e+01,0.258800,427.816000,66.680000,85.027320,3831.000000,1.278000e+03,3.365000e+03
50%,2009.000000,8.899196e+03,5.798054e+03,4.903994e+03,28.000000,28.000000,3.750000e+02,4.000000e+01,13.600000,45.000000,...,80.042999,287.000000,2.560000e+02,0.629670,1471.182780,438.280000,351.621740,7492.000000,5.652000e+03,7.970000e+03
75%,2014.000000,2.762698e+04,1.660771e+04,1.418657e+04,290.000000,280.000000,4.020250e+03,6.340000e+02,25.300000,112.000000,...,83.460297,2644.750000,1.524000e+03,1.399090,3260.092010,5321.600000,708.915880,15319.000000,2.376200e+04,2.334200e+04
max,2019.000000,1.433784e+06,1.230020e+06,1.007108e+06,699412.000000,669877.000000,1.956422e+07,8.572143e+06,2889.000000,2386.000000,...,100.000000,313052.000000,1.393815e+06,4.952780,8065.887290,528263.250000,3766.861460,238246.000000,7.127191e+06,2.104411e+06


In [563]:
df.isna().sum()
df[df['Firms that spend on R&D (% of firms)'].notna()]

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
Country code,,,,,,,,,,,,,,,,,,,,,
32,Yes,No,No,ARG,Argentina,2006,39289.876,28338.514,22023.772,1627,...,84.953201,4597.0,1020.0,0.45216,891.83275,5587.62,207.45801,19428.0,58283.0,77711.0
68,Yes,No,No,BOL,Bolivia (Plurinational State of),2006,9395.449,5697.098,4201.358,78,...,85.960503,NaN,NaN,NaN,NaN,68.47,NaN,3677.0,1688.0,5365.0
152,Yes,No,No,CHL,Chile,2006,16354.507,11582.020,9651.707,776,...,86.185699,2924.0,291.0,NaN,NaN,3112.32,NaN,NaN,NaN,31577.0
170,Yes,No,No,COL,Colombia,2006,43200.901,27496.608,22310.119,365,...,NaN,1861.0,142.0,0.14990,NaN,1401.91,NaN,8400.0,13270.0,21670.0
218,Yes,No,No,ECU,Ecuador,2006,14059.379,8197.648,6769.626,92,...,85.240097,751.0,8.0,0.12863,70.05997,126.89,18.70637,2387.0,11034.0,13421.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,Yes,No,No,TJK,Tajikistan,2019,9321.023,5679.169,4218.612,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
792,Yes,No,No,TUR,Turkey,2019,83429.607,57564.209,49629.023,555,...,79.155502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
804,Yes,No,No,UKR,Ukraine,2019,43993.643,51147.771,32789.043,27,...,67.694504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [564]:
df_saturated = df[(df['Subject'] == 'All') & (df['Year'] <= 2017) & (df['Year'] >= 2007)].iloc[:, 0:26].dropna()

df_saturated.corr()

,Year,Population,Population 25 Years Before,Population 25+,Documents,Citable documents,Citations,Self-citations,Citations per Document,H index,Avg. Adult Schooling,GDP per capita (World Bank),GDP per capita (PWT),MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
Year,1.000000,0.009922,0.014714,0.015338,0.047561,0.046495,-0.050746,-0.029243,-0.320288,-0.001244,0.093079,0.025919,0.029829,0.025549,0.026188,0.109591,0.103341,-0.079708,-0.095976,0.006187
Population,0.009922,1.000000,0.991790,0.989766,0.552651,0.587161,0.357233,0.422666,-0.082699,0.291788,-0.051758,-0.062134,-0.058013,0.946209,0.943012,-0.057336,-0.053489,-0.064246,-0.061746,0.007179
Population 25 Years Before,0.014714,0.991790,1.000000,0.999590,0.597882,0.635416,0.388486,0.458150,-0.082272,0.318646,-0.018135,-0.041516,-0.037171,0.974259,0.971932,-0.042910,-0.036060,-0.052774,-0.048666,0.029680
Population 25+,0.015338,0.989766,0.999590,1.000000,0.607355,0.645012,0.396847,0.467528,-0.080457,0.324027,-0.015015,-0.034338,-0.029906,0.976070,0.974702,-0.035995,-0.030342,-0.047260,-0.044028,0.036416
Documents,0.047561,0.552651,0.597882,0.607355,1.000000,0.997716,0.892853,0.895580,-0.010987,0.772828,0.249831,0.225952,0.243304,0.734841,0.745598,0.220595,0.251791,0.177109,0.193771,0.297607
Citable documents,0.046495,0.587161,0.635416,0.645012,0.997716,1.000000,0.879772,0.887527,-0.014842,0.762416,0.244903,0.219806,0.236674,0.768521,0.779033,0.213901,0.244541,0.170965,0.187251,0.293858
Citations,-0.050746,0.357233,0.388486,0.396847,0.892853,0.879772,1.000000,0.969272,0.068924,0.767586,0.257861,0.244582,0.261457,0.527600,0.538126,0.238672,0.271018,0.255407,0.277121,0.309769
Self-citations,-0.029243,0.422666,0.458150,0.467528,0.895580,0.887527,0.969272,1.000000,0.035837,0.640659,0.174310,0.164795,0.177603,0.592493,0.604062,0.133560,0.154828,0.140436,0.154980,0.203331
Citations per Document,-0.320288,-0.082699,-0.082272,-0.080457,-0.010987,-0.014842,0.068924,0.035837,1.000000,0.066881,-0.042091,0.025303,0.022934,-0.081503,-0.078462,0.086702,0.095628,0.289018,0.309062,0.075283
H index,-0.001244,0.291788,0.318646,0.324027,0.772828,0.762416,0.767586,0.640659,0.066881,1.000000,0.524064,0.478715,0.502240,0.438674,0.444597,0.583693,0.644629,0.545129,0.577193,0.632840


In [565]:
df_saturated.drop(columns=['GDP per capita (PWT)'], inplace=True)
df_saturated



,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Subject,Avg. Adult Schooling,GDP per capita (World Bank),MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
Country code,,,,,,,,,,,,,,,,,,,,,
8,Yes,No,No,ALB,Albania,2007,3033.993,2790.284,1723.381,83,...,All,9.2,8754.569513,25670.6128,15855.1052,0.003233,0.005235,0.062628,0.101400,-0.406624
12,Yes,No,No,DZA,Algeria,2007,34166.976,20452.901,16968.225,2051,...,All,7.2,12570.654703,147260.8872,122171.2200,0.013928,0.016788,0.186770,0.225126,-0.565725
24,Yes,No,No,AGO,Angola,2007,20905.360,8952.971,6929.155,36,...,All,4.5,5443.117933,40288.3695,31181.1975,0.000894,0.001155,0.010553,0.013635,-1.206412
28,Yes,Yes,No,ATG,Antigua and Barbuda,2007,84.029,61.780,48.763,10,...,All,9.2,24408.181522,568.3760,448.6196,0.017594,0.022291,0.100286,0.127056,0.405212
32,Yes,No,No,ARG,Argentina,2007,39684.303,28794.550,22353.570,8227,...,All,8.9,17900.706342,256271.4950,198946.7730,0.032103,0.041353,0.765970,0.986677,-0.015959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,Yes,No,No,URY,Uruguay,2017,3436.645,3154.459,2209.904,1595,...,All,8.7,20551.409420,27443.7933,19226.1648,0.058119,0.082960,0.370217,0.528454,0.431783
860,Yes,No,No,UZB,Uzbekistan,2017,31959.774,21375.114,17145.037,615,...,All,11.5,6253.103860,245813.8110,197167.9255,0.002502,0.003119,0.010783,0.013444,-0.557881
704,Yes,No,No,VNM,Viet Nam,2017,94600.643,70883.488,58172.369,6642,...,All,8.2,6171.884192,581244.6016,477013.4258,0.011427,0.013924,0.079762,0.097190,0.009443


In [566]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt 
from sklearn.preprocessing import  OneHotEncoder
from sklearn.preprocessing import StandardScaler

hot_enc = OneHotEncoder(sparse=False)

df_test = df_saturated[['Year', 'GDP per capita (World Bank)', 'Documents / M-TAMS', 'Citations / M-TAMS']]
# df_test = df_test.groupby(by=[df_saturated.index, 'Year']).agg({'GDP per capita (World Bank)': 'mean', 'Documents / M-TAMS': 'mean', 'H index': 'max', 'Gov. Effectiveness': 'mean' })

X = df_test.iloc[:, 2:]
y = df_test.iloc[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# scaler.fit(X_train)
# scaler.transform(X_train)


lr = LinearRegression(normalize=True)

lr.fit(X_train,y_train)


LinearRegression(normalize=True)

In [567]:
lr.score(X_test, y_test)

0.6529713413421514